In [5]:
import pandas as pd
from pandas import DataFrame
from pandas import ExcelWriter
import requests
from bs4 import BeautifulSoup

# KRX에서 회사명/업종/주요제품에 대한 정보 가져오기 
code_df = pd.read_html('http://kind.krx.co.kr/corpgeneral/corpList.do?method=download&searchType=13', header=0)[0]
#code_df.종목코드 = code_df.종목코드.map("{:06d}".format)
#code_df = code_df[['종목코드', '회사명']]
#code_df = code_df.rename(columns={'종목코드': 'code', '회사명':'name'})
#code_list = code_df['code'].values.tolist()

# 한국신용평가에서 BBB- 등급 채권 5년 수익률(회사채 수익률)가져오기
bond_df = pd.read_html('https://www.kisrating.com/ratingsStatistics/statics_spread.do#', header=0)[0]
discount_rate = bond_df['5년'][10]

In [8]:
code_df['주요제품']

0                       고무벨트(V벨트,콘베이어벨트,평벨트),프라스틱제품 제조,판매
1                                             지주회사/부동산 임대
2       수출입업(시멘트,철강금속,전기전자,섬유,기계화학),상품중개,광업,채석업/하수처리 서...
3                                  외주주택, 자체공사, 일반건축, 토목 등
4                           정기,부정기,전용선,콘테이너선,자동차선,LNG선 해운
                              ...                        
2453        ILS(종합군수지원), IETM(전자식기술교범), CBT(전자식교보재), 기술번역
2454    유전자가위 (유전체 교정 도구) 및 이를 이용한 유전자세포치료제, 고부가가치 가축,...
2455                      Cloud Solution (D,A,E,Rstation)
2456                                     강아지암치료제,돼지DNA테라피
2457                                       은나노스텝, 닥터오렌지 등
Name: 주요제품, Length: 2458, dtype: object

In [7]:
def get_fnguide_table_Y(code):
    try: 
        url = requests.get('http://comp.fnguide.com/SVO2/ASP/SVD_main.asp?pGB=1&gicode=A%s'%(code))
        url = url.content
        html = BeautifulSoup(url,'html.parser')
        body = html.find('body')
        fn_body = body.find('div',{'class':'fng_body asp_body'})
        ur_table = fn_body.find('div',{'id':'div15'})
        table = ur_table.find('div',{'id':'highlight_D_Y'})
        tbody = table.find('tbody')
        tr = tbody.find_all('tr')
        Table = DataFrame()
 
        for i in tr:
            category = i.find('span',{'class':'txt_acd'})
 
            if category == None:
                category = i.find('th')   
 
            category = category.text.strip()
            value_list =[]
 
            j = i.find_all('td',{'class':'r'})
 
            for value in j:
                temp = value.text.replace(',','').strip() 
                try:
                    temp = float(temp)
                    value_list.append(temp)
                except:
                    value_list.append(0)
 
            Table['%s'%(category)] = value_list
            thead = table.find('thead')
            tr_2 = thead.find('tr',{'class':'td_gapcolor2'}).find_all('th')
            year_list = []
            for i in tr_2:
                try:
                    temp_year = i.find('span',{'class':'txt_acd'}).text
                except:
                    temp_year = i.text
                year_list.append(temp_year)
            Table.index = year_list

        Table.reset_index(level=0, inplace=True)
        Table = Table.rename(columns={'index': 'year'}) 
        Table['code'] = code
        return Table
    
    except: 
        print('error detection!')

In [57]:
def get_fnguide_table_Q(code):
    
    try: 
        url = requests.get('http://comp.fnguide.com/SVO2/ASP/SVD_main.asp?pGB=1&gicode=A%s'%(code))
        url = url.content
 
        html = BeautifulSoup(url,'html.parser')
        body = html.find('body')
 
        fn_body = body.find('div',{'class':'fng_body asp_body'})
        ur_table = fn_body.find('div',{'id':'div15'})
        table = ur_table.find('div',{'id':'highlight_D_Q'})
 
        tbody = table.find('tbody')
        tr = tbody.find_all('tr')
        Table = DataFrame()
 
        for i in tr:
            category = i.find('span',{'class':'txt_acd'})
 
            if category == None:
                category = i.find('th')   
 
            category = category.text.strip()
            value_list =[]
            j = i.find_all('td',{'class':'r'})
            for value in j:
                temp = value.text.replace(',','').strip()
 
                try:
                    temp = float(temp)
                    value_list.append(temp)
                except:
                    value_list.append(0)
            Table['%s'%(category)] = value_list  
            thead = table.find('thead')
            tr_2 = thead.find('tr',{'class':'td_gapcolor2'}).find_all('th')
            year_list = []
 
            for i in tr_2:
                try:
                    temp_year = i.find('span',{'class':'txt_acd'}).text
                except:
                    temp_year = i.text
                year_list.append(temp_year)
            Table.index = year_list
 
        Table.reset_index(level=0, inplace=True)
        Table = Table.rename(columns={'index': 'year'}) 
        Table['code'] = code
        return Table
    
    except: 
        print('error detection!')

In [218]:
def get_price(code):
    try: 
        ''' 경로 탐색'''
        url = requests.get('http://comp.fnguide.com/SVO2/ASP/SVD_main.asp?pGB=1&gicode=A%s'%(code))
        url = url.content

        html = BeautifulSoup(url,'html.parser')
        body = html.find('body')

        fn_body = body.find('div',{'class':'fng_body'})
        ur_table = fn_body.find('div', {'id':'div1'})
        table = ur_table.find('div', {'class':'um_table'})

        tbody = table.find('tbody')
        tr = tbody.find_all('tr')
        share_price = tr[0]
        price = share_price.find('td', {'class':'r'})
        current_price = price.text.split('/')[0].replace(',','').strip()
        
        return current_price
    
    except:
        print('error detection!')

In [246]:
def get_share_quantity(code):
    try:
        url = requests.get('http://comp.fnguide.com/SVO2/ASP/SVD_main.asp?pGB=1&gicode=A%s'%(code))
        url = url.content

        html = BeautifulSoup(url,'html.parser')
        body = html.find('body')

        fn_body = body.find('div',{'class':'fng_body'})
        ur_table = fn_body.find('div', {'id':'div1'})
        table = ur_table.find('div', {'class':'um_table'})

        tbody = table.find('tbody')
        tr = tbody.find_all('tr')
        stock_quantity = tr[6]
        quantity = stock_quantity.find('td', {'class':'r'})
        quantity = quantity.text.split('/')[0].replace(',','').strip()

        
        fn_body2 = body.find('div',{'class':'fng_body'})
        ur_table2 = fn_body2.find('div', {'id':'div4'})
        table2 = ur_table2.find('div', {'id':'svdMainGrid5'})

        tbody2 = table2.find('tbody')
        tr2 = tbody2.find_all('tr')
        stock_quantity2 = tr2[4]

        self_quantity = stock_quantity2.find_all('td', {'class':'r'})[1].text.replace(',','').strip()
        
        if self_quantity == '':
            all_quantity = int(quantity)
        else:
            all_quantity = int(quantity) - int(self_quantity)


        return all_quantity
    except:
        print('error detection!')

In [220]:
# 전체 주식에 대해 시험해보고 싶을 때 
def all_stock_yearly_info(code):
    df = pd.DataFrame(columns = [])
    for n, i in enumerate(code_list):    
        df = df.append(get_fnguide_table_Y(i))
        print(n)

In [361]:
table_Y = get_fnguide_table_Y('000660')
equity_capital = table_Y['지배주주지분'][4] # 자기자본
first, second, third = table_Y['ROE'][2:5]
weighted_average = (3*third + 2*second + 1*first) / 6



exceeded_profit = (equity_capital * 1000000) * ((weighted_average - discount_rate))
enterprise_value = ((equity_capital * 1000000) + exceeded_profit / discount_rate) * 100

enterprise_value /get_share_quantity('000660')

115921.49567312194

In [362]:
def S_RIM_Y(code, discount_rate):
    try:
        table_Y = get_fnguide_table_Y(code)
        equity_capital = table_Y['지배주주지분'][4] # 자기자본
        first, second, third = table_Y['ROE'][2:5]
        weighted_average = (3*third + 2*second + 1*first) / 6
        exceeded_profit = (equity_capital * 1000000) * ((weighted_average - discount_rate))
        enterprise_value = ((equity_capital * 1000000) + exceeded_profit / discount_rate) * 100
        valuation = enterprise_value / get_share_quantity(code)
        
        return valuation
    
    except:
        print('error detection!')

In [371]:
def S_RIM_Q(code, discount_rate):
    try:
        table_Q = get_fnguide_table_Q(code)
        equity_capital = table_Q['지배주주지분'][4] # 자기자본
        first, second, third = table_Q['ROE'][2:5]
        weighted_average = (3*third + 2*second + 1*first) / 6
        exceeded_profit = (equity_capital * 1000000) * ((weighted_average - discount_rate))
        enterprise_value = ((equity_capital * 1000000) + exceeded_profit / discount_rate) * 100
        valuation = enterprise_value / get_share_quantity(code)
        
        return valuation
    
    except:
        print('error detection!')